# Imports

In [1]:
#Enable matplotlib to display in jupyter notebook & import it
%matplotlib inline

import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import re

# Reading Files

In [2]:
#Read Files
#"originals" should never be modified.  They exist to check work.  They should be deleted in our final work
listings_original = pd.read_csv('data/listings.csv')
calendar_oiginal = pd.read_csv('data/calendar.csv')
reviews_original = pd.read_csv('data/reviews.csv')

listings = listings_original.copy()
calendar = calendar_oiginal.copy()
reviews = reviews_original.copy()

# #Final version should just have the code below
# listings = pd.read_csv('data/listings.csv')
# calendar = pd.read_csv('data/calendar.csv')
# reviews = pd.read_csv('data/reviews.csv')

#3585

# Clean the Data

## Listings Data

In [3]:
#for testing
listings = listings_original.copy()

#listings.columns[(listings.isnull().sum()==3585)]
#returns these (empty variables)
#Index(['neighbourhood_group_cleansed', 'has_availability', 'license','jurisdiction_names'],
      
      
# Description:  
# Original shape = (3585, 95)

# id
# Type: Float
# What: The listing id is a unique id # for each property being listed on airBnB
# Cleaning: change to string, there is no reason to treat these as numbers, SET AS INDEX
listings.id = listings.id.astype(str)
#listings.index = set_index('id', inplace=True) #this also sets index but doesn't leave me with id as a column
listings.index = listings.id.copy()

# listing_url
# Type: String
# What: Url in for the listing in the format: "https://www.airbnb.com/rooms/" + listing.id
# Cleaning: leave in for easy access to listings while exploring data

# scrape_id
# Type: int
# What: Identify which webscrape the data came from, all of Boston came from the same scrape: 20160906204935
#Cleaning: Delete.  Not relevant to our analysis
del(listings['scrape_id'])

# last_scraped
# Type: str
# What: The date that our data was scraped, it is the same for all of Boston entries: 2016-09-07
# Cleaning : Convert to date
listings.last_scraped = pd.to_datetime(listings.last_scraped)

# name
# Type: str
# What: This text is the "title" of any listing.  In a search result it would 
#       be the text that users see as the first description of a listing.  It
#       is also the title of the page for a listing in browser history. etc.
# Cleaning:  None
# Uses: keyword analysis
pass

# summary
# Type: str or NaN
# What: Prose that is displayed on listing webpage for "About this listing".
# Cleaning: Fill NaN with "none" so that all types match
# Uses: Keyword analysis
pass
len(listings.summary.unique()) #=3114
listings.summary = listings.summary.fillna('none')
listings.summary[listings.summary == 'none'].count() #=143
listings.summary.value_counts().index[1] #most common summary other than "none"
listings[listings.summary == listings.summary.value_counts().index[1]]
#more exploration of duplicates needed


# space
# Type: str or NaN
# What: [OPTIONAL] Prose to describe the inside space of a listing
# Cleaning: Fill NaN with "none" so that all types match

listings.space.isnull().any()
len(listings.space.unique()) #=2269
listings.space = listings.space.fillna('none')
listings.space[listings.space == 'none'].count() #=1057
listings.space.value_counts()
#more exploration of duplicates needed

# description
# Type: str
# What: Prose that contains the first 1000 characters of the merging of other descriptive prose:
#     summary
#     space
#     experiences_offered
#     neighborhood_overview
#     notes
#     transit
#     access
#     interaction
#     house_rules
# Cleaning: Delete.  the fact that it only gives us the first 1000 characters makes it unuseful.
del(listings['description'])

pass

# experiences_offered
# Type: str
# What: 'none' for every entry
# Cleaning: Delete
del(listings['experiences_offered'])

# neighborhood_overview
# Type: str or NaN
# What: [OPTIONAL] description by host of the neighborhood 
# Cleaning: Fill NaN with "none" so that all types match
listings.neighborhood_overview.isnull().sum() #=1415
len(listings.neighborhood_overview.unique()) #=1729
listings.neighborhood_overview = listings.neighborhood_overview.fillna('none')
#more exploration of duplicates needed

# notes
# Type: str or NaN
# What: [OPTIONAL] under "Other things to note" in the description on webpage
# Cleaning: Fill NaN with "none" so that all types match
listings.notes.isnull().sum() #=1975
listings.notes.fillna('none')
len(listings.notes.unique()) #=1270
#more exploration of duplicates needed


# transit
# Type: str or NaN
# What: [OPTIONAL] under "Getting around" in the description on webpage
# Cleaning: Fill NaN with "none" so that all types match
listings.transit.isnull().sum()
listings.transit.fillna('none')
len(listings.transit.unique())
#more exploration of duplicates needed


# access
# Type: str or NaN
# What: [OPTIONAL] under "Guest access" in the description on webpage
# Cleaning: Fill NaN with "none" so that all types match
listings.access.isnull().sum() #=1489
listings.access.fillna('none') 
len(listings.access.unique()) #=1763
#more exploration of duplicates needed


# interaction
# Type: str or NaN
# What: [OPTIONAL] under "Interaction with guests" in the description on webpage
# Cleaning: Fill NaN with "none" so that all types match
listings.interaction.isnull().sum() #=1554
listings.interaction.fillna('none') 
len(listings.interaction.unique()) #=1618
#more exploration of duplicates needed


# house_rules
# Type: str or NaN
# What: [OPTIONAL] under "Interaction with guests" in the description on webpage
# Cleaning: Fill NaN with "none" so that all types match
listings.house_rules.isnull().sum() #=1192
listings.house_rules.fillna('none') 
len(listings.house_rules.unique()) #=1929



# thumbnail_url
# medium_url
# picture_url
# xl_picture_url
# Type: str or NaN
# What: URL to different image resourses
# Cleaning: Delete
del(listings['thumbnail_url'])
del(listings['medium_url'])
del(listings['picture_url'])
del(listings['xl_picture_url'])

# host_id
# Type: int
# What: The listing id is a unique id # for each host on airbnb
# Cleaning: change to string, there is no reason to treat these as numbers
listings.host_id = listings.host_id.astype(str)

# host_url
# Cleaning: Delete
del(listings['host_url'])


# host_name
# Type: str
# What: Host first name only
# Cleaning: append the user id to each host_Name to distinguish between people with the same name
listings['host_name'] = listings['host_name'] + ' ' + listings['host_id']
#listings.host_name.value_counts()

# host_since
# Type: str
# What:
# Cleaning: convert to date
listings.host_since = pd.to_datetime(listings.host_since)

# host_location
# Type: str
# What: Location that the host lives in
# Cleaning: need to parse out city, state, country if they are there ###############
listings.host_location[1]


# host_about

# host_response_time

# host_response_rate

# host_acceptance_rate

# host_is_superhost

# host_thumbnail_url
# Cleaning: Delete
del(listings['host_thumbnail_url'])

# host_picture_url
# Cleaning: Delete
del(listings['host_picture_url'])

# host_neighbourhood

# host_listings_count
# Type: int
# What: The number of listings that the host has ACROSS ALL OF AIRBNB
# Related: If you want the number of listings that they have in this set, use calculated_host_listings_count

# host_total_listings_count
# Cleaning: Delete
(listings['host_total_listings_count'] == listings['host_listings_count']).all() #true
del(listings['host_total_listings_count'])

# host_verifications

# host_has_profile_pic
# Cleaning: Delete, only 7 don't have a pic, also not relevant to our work
listings.host_has_profile_pic[listings.host_has_profile_pic == 'f'] #there were 7 listings
del(listings['host_has_profile_pic'])

# host_identity_verified


# street

# neighbourhood
# neighbourhood_cleansed
n = listings[['neighbourhood','neighbourhood_cleansed']]
n = n[(n['neighbourhood'] != n['neighbourhood_cleansed'])]
n[(n['neighbourhood'] == n['neighbourhood'])]
########it is strange to me that these don't match up.  there's something here we might want to understand

# neighbourhood_group_cleansed
# Cleaning: Delete, empty variable (all null)
del listings['neighbourhood_group_cleansed']


# city
# state
# zipcode


# market
# Cleaning: Delete
listings.market[listings.market != 'Boston'] ######the results from this are very odd
del listings['market']


# smart_location
# Cleaning: Delete, redundant with city
listings[['smart_location','city']] #there are only two listings that don't appear to match, 14767573 and 14743129
del listings['city']


# country_code
# Cleaning: Delete
(listings.country_code == listings.country_code[0]).all() #=True: 'US'
del listings['country_code']

# country
(listings.country == listings.country[0]).all() #=True: 'United States'
del listings['country']

# latitude
# longitude
# is_location_exact
# property_type
# room_type
# accommodates
# bathrooms
# bedrooms
# beds
# bed_type
# amenities

# square_feet
# Cleaning:  We may want to consider deleting this variable, only 56 entries have data
listings.square_feet.isnull().sum() #=3529
listings.square_feet = listings.square_feet.replace('[^0-9.]+','',regex=True).astype(float)

# price
#this regex "replaces" anything that is not a digit or a decimal with the empty string
#effectively removing anything that's not part of the number
listings.price = listings.price.replace('[^0-9.]+','',regex=True).astype(float)


# weekly_price
listings.weekly_price = listings.weekly_price.replace('[^0-9.]+','',regex=True).astype(float)
# monthly_price
listings.monthly_price = listings.monthly_price.replace('[^0-9.]+','',regex=True).astype(float)
# security_deposit
listings.security_deposit = listings.security_deposit.replace('[^0-9.]+','',regex=True).astype(float)
# cleaning_fee
listings.cleaning_fee = listings.cleaning_fee.replace('[^0-9.]+','',regex=True).astype(float)

# guests_included
# extra_people
# minimum_nights
# maximum_nights
# calendar_updated

# has_availability
# Cleaning: Delete, empty variable (all null)
del listings['has_availability']



# availability_30
# availability_60
# availability_90
# availability_365
# calendar_last_scraped
# number_of_reviews
# first_review
# last_review
# review_scores_rating
# review_scores_accuracy
# review_scores_cleanliness
# review_scores_checkin
# review_scores_communication
# review_scores_location
# review_scores_value
# requires_license

# license
# Cleaning: Delete, empty variable (all null)
del listings['license']

# jurisdiction_names
# Cleaning: Delete, empty variable (all null)
del listings['jurisdiction_names']

# instant_bookable
# cancellation_policy
# require_guest_profile_picture
# require_guest_phone_verification


# calculated_host_listings_count
# Type: int
# What: The number of listings that the host has THIS data set
# Related: If you want the number of th listings that the host has across ALL of airbnb
#          use host_listings_count
#listings.calculated_host_listings_count == listings.host_listings_count # not all true
#listings[['host_id','calculated_host_listings_count','host_listings_count']][listings.calculated_host_listings_count != listings.host_listings_count]


# reviews_per_month



#####dead end code to fix zipcodes
listings[listings.zipcode.isnull()].neighbourhood_cleansed
nczdict = {}
for n in listings.neighbourhood_cleansed.unique():
    nczdict[n] = listings[listings.neighbourhood_cleansed == n].zipcode.value_counts().idxmax()

#print(nczdict)

nzdict = {}
for n in listings.neighbourhood.unique():
    if type(n) == str:
        nzdict[n] = listings[listings.neighbourhood == n].zipcode.value_counts().idxmax()

#print(nzdict)


#######



In [13]:
#####
# This code is a work in progress to fil in missing zip codes
#
####
from geopy.geocoders import Nominatim

def latlonToZip(lat, lon):
    geolocator = Nominatim()
    try:
        location = geolocator.reverse(str(lat)+','+str(lon))
        z = re.compile('(,\s)([0-9]{5})(,\sUnited)')
        #print(z.findall(location[0])[0][1])
        return z.findall(location[0])[0][1]
    except:
        print(str(lat)+','+str(lon))
        return np.nan
    
    print(str(lat)+','+str(lon))
    return np.nan


#lat = listings[listings.zipcode.isnull()][['latitude','longitude']].latitude[0]
#lon = listings[listings.zipcode.isnull()][['latitude','longitude']].longitude[0]

temp = listings[listings.zipcode.isnull()].copy()
temp.shape
#latlonToZip(lat,lon)
temp['new_zip'] = temp.apply(lambda x: latlonToZip(x['latitude'], x['longitude']), axis=1)

listings['new_zip'] = listings['zipcode'].copy()

42.34128360140666,-71.08567288767645


In [33]:
final = listings.combine_first(temp)
final[['zipcode','new_zip']][final.zipcode != final.new_zip]

,zipcode,new_zip
id,,
10084216,NaN,02116
11229070,NaN,02113
1141888,NaN,02115
12026868,NaN,02134
12646288,NaN,02128
12789527,NaN,02130
12971157,NaN,02128
13073078,NaN,NaN
13077603,NaN,02119


In [232]:
#coerce appropriate fields to datetime
listings['host_since'] = pd.to_datetime(listings['host_since'])

listings['price'] = listings['price'].str.replace('$','')
listings['price'] = listings['price'].str.replace(',','')
listings['price'] = listings['price'].astype(float)

listings['zipcode'] = listings['zipcode'].str.replace('-','')
listings['zipcode'] = listings['zipcode'].str.replace(' ', '')
listings['mainzip'] = pd.Series([str(z)[:5] for z in listings['zipcode'].fillna('00000')]) ##THIS IS NOT WORKING AS I WANT IT TO; AIMING TO GRAB FIRST FIVE NUMBERS TO CREATE NEW FILE

#CLEANING FEE

listings['cleaning_fee'] = listings['cleaning_fee'].str.replace('$','')
listings['cleaning_fee'] = listings['cleaning_fee'].str.replace(',','')

listings['cleaning_fee'] = listings['cleaning_fee'].astype(float)

#CALENDAR LAST SCRAPED
listings['calendar_last_scraped'] = pd.to_datetime(listings['calendar_last_scraped'])


##################################      REVIEWS       #####################################################
reviews['date'] = pd.to_datetime(reviews['date'])

##################################      CALENDAR       #####################################################
calendar['date'] = pd.to_datetime(calendar['date'])
calendar['listing_id'] = calendar['listing_id'].astype('category')
calendar['price'] = calendar['price'].str.replace('$','')
calendar['price'] = calendar['price'].str.replace(',','')
calendar['price'].fillna(0)
calendar['price'] = calendar['price'].astype(float)



AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [5]:
listings_original['amenities']

0       {TV,"Wireless Internet",Kitchen,"Free Parking ...
1       {TV,Internet,"Wireless Internet","Air Conditio...
2       {TV,"Cable TV","Wireless Internet","Air Condit...
3       {TV,Internet,"Wireless Internet","Air Conditio...
4       {Internet,"Wireless Internet","Air Conditionin...
5       {"Cable TV","Wireless Internet","Air Condition...
6       {TV,Internet,"Wireless Internet",Kitchen,"Free...
7       {TV,Internet,"Wireless Internet","Air Conditio...
8       {"Wireless Internet","Pets live on this proper...
9       {TV,"Cable TV",Internet,"Wireless Internet","A...
10      {TV,"Wireless Internet","Air Conditioning",Kit...
11      {TV,"Cable TV",Internet,"Wireless Internet","A...
12      {"Cable TV","Wireless Internet","Air Condition...
13      {TV,Internet,"Wireless Internet","Air Conditio...
14      {Internet,"Wireless Internet",Kitchen,"Free Pa...
15      {TV,Internet,"Wireless Internet","Air Conditio...
16      {TV,"Cable TV",Internet,"Wireless Internet",Ki...
17      {TV,"W

In [6]:
l2 = listings_original['amenities'].map(
    lambda amns: "|".join([amn.replace("}", "").replace("{", "").replace('"', "")\
                           for amn in amns.split(",")]))
listings_original['amenities'].map(
    lambda amns: "|".join([amn.replace("}", "").replace("{", "").replace('"', "")\
                           for amn in amns.split(",")]))

0       TV|Wireless Internet|Kitchen|Free Parking on P...
1       TV|Internet|Wireless Internet|Air Conditioning...
2       TV|Cable TV|Wireless Internet|Air Conditioning...
3       TV|Internet|Wireless Internet|Air Conditioning...
4       Internet|Wireless Internet|Air Conditioning|Ki...
5       Cable TV|Wireless Internet|Air Conditioning|Ki...
6       TV|Internet|Wireless Internet|Kitchen|Free Par...
7       TV|Internet|Wireless Internet|Air Conditioning...
8       Wireless Internet|Pets live on this property|C...
9       TV|Cable TV|Internet|Wireless Internet|Air Con...
10      TV|Wireless Internet|Air Conditioning|Kitchen|...
11      TV|Cable TV|Internet|Wireless Internet|Air Con...
12      Cable TV|Wireless Internet|Air Conditioning|Ki...
13      TV|Internet|Wireless Internet|Air Conditioning...
14      Internet|Wireless Internet|Kitchen|Free Parkin...
15      TV|Internet|Wireless Internet|Air Conditioning...
16      TV|Cable TV|Internet|Wireless Internet|Kitchen...
17      TV|Wir

In [7]:
np.unique(np.concatenate(l2.map(lambda amns: amns.split("|"))))[1:]

array(['24-Hour Check-in', 'Air Conditioning', 'Breakfast',
       'Buzzer/Wireless Intercom', 'Cable TV', 'Carbon Monoxide Detector',
       'Cat(s)', 'Dog(s)', 'Doorman', 'Dryer', 'Elevator in Building',
       'Essentials', 'Family/Kid Friendly', 'Fire Extinguisher',
       'First Aid Kit', 'Free Parking on Premises',
       'Free Parking on Street', 'Gym', 'Hair Dryer', 'Hangers', 'Heating',
       'Hot Tub', 'Indoor Fireplace', 'Internet', 'Iron', 'Kitchen',
       'Laptop Friendly Workspace', 'Lock on Bedroom Door', 'Other pet(s)',
       'Paid Parking Off Premises', 'Pets Allowed',
       'Pets live on this property', 'Pool', 'Safety Card', 'Shampoo',
       'Smoke Detector', 'Smoking Allowed', 'Suitable for Events', 'TV',
       'Washer', 'Washer / Dryer', 'Wheelchair Accessible',
       'Wireless Internet', 'translation missing: en.hosting_amenity_49',
       'translation missing: en.hosting_amenity_50'], 
      dtype='<U42')

In [8]:
listings['amenities'] = listings['amenities'].map(
    lambda amns: "|".join([amn.replace("}", "").replace("{", "").replace('"', "")\
                           for amn in amns.split(",")]))
                           
amenities = np.unique(np.concatenate(listings['amenities'].map(lambda amns: amns.split("|"))))[1:]
np.array([listings['amenities'].map(lambda amns: amn in amns) for amn in amenities[0:43]])

array([[False, False, False, ..., False, False, False],
       [False,  True,  True, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ..., 
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [ True,  True,  True, ..., False, False,  True]], dtype=bool)

In [14]:
###################################      LISTINGS      ##################################################
#Rob: Here I borrowed code from kaggle, cleaning up the junk in the amenities list, removing parentheses, etc. first
#Then I isolated each unique amenity and created column t/f data frames for them, and then eventually concatenated 
#with original listings data to build features. Just see the features df

#Dummy Variables in Listings
listings['amenities'] = listings['amenities'].map(
    lambda amns: "|".join([amn.replace("}", "").replace("{", "").replace('"', "")\
                           for amn in amns.split(",")]))
                           
amenities = np.unique(np.concatenate(listings['amenities'].map(lambda amns: amns.split("|"))))[1:]
amenity_arr = np.array([listings['amenities'].map(lambda amns: amn in amns) for amn in amenities[0:43]])

#Listing exploratory
#listings['amenities'].map(lambda amns: amns.split("|")).head() #test print?
np.unique(np.concatenate(listings['amenities'].map(lambda amns: amns.split("|"))))[1:]

#Merged dataset with listings and dummy variables
features = pd.concat([listings, pd.DataFrame(data=amenity_arr.T, columns=amenities[0:43])], axis=1)
#features


In [15]:
#Rob: groupby'ed the calendar listing IDs for total rev, count of bookings/bookigns, and then revenue / booking
###########################      CALENDAR SUMMARY FOR MERGING       #########################################
#cleanCal = calendar[calendar['available'] == 't']

#cleanCal = calendar.groupby('listing_id', as_index = True)['price'].sum().sort_values(['price'], ascending = False)
#cleanCal

# cleanCal.columns = ['listing_id', 'total_revenue']
# cleanCal2 = calendar.groupby('listing_id', as_index = False)['price'].count().sort_values(['price'], ascending = False)
# cleanCal2.columns = ['listing_id', 'num_bookings']
# cleanCal = cleanCal.merge(cleanCal2)
# cleanCal['total_revenue'] = cleanCal['total_revenue'].fillna(0)
# cleanCal['revenue_per_booking'] = cleanCal['total_revenue']/cleanCal['num_bookings']
# cleanCal['revenue_per_booking'] = cleanCal['revenue_per_booking'].fillna(0)
# cleanCal

In [16]:
#Rob: groupby'ed the calendar listing IDs for total rev, count of bookings/bookigns, and then revenue / booking
###########################      CALENDAR SUMMARY FOR MERGING       #########################################
available_cal = calendar[calendar['available'] == 't']
mean_price_per_night = available_cal.groupby('listing_id', as_index=False)['price'].mean()

occupied_cal = calendar[calendar['available'] == 'f']
occupied_cal.groupby('listing_id', as_index=False).head()


ex = calendar[calendar['listing_id'] == 3075044].sort_values(['date'])
ex['day_of_week'] = [d.dayofweek for d in ex['date']]
ex_available = ex[ex['available'] == 't']
ex_mean_prices = ex_available.groupby('day_of_week', as_index=True)['price'].mean()
ex = ex.price.fillna(value = ex.day_of_week.apply(lambda x: ex_mean_prices[int(x)]))
ex
calendar[calendar['listing_id'] == 3075044].sort_values(['date'])



#from datetime import date
#[d.dayofweek for d in calendar[calendar['listing_id'] == 3075044]['date']]
# cleanCal = calendar.groupby('listing_id', as_index = False)['price'].sum().sort_values(['price'], ascending = False)
# cleanCal

# cleanCal.columns = ['listing_id', 'total_revenue']
# cleanCal2 = calendar.groupby('listing_id', as_index = False)['price'].count().sort_values(['price'], ascending = False)
# cleanCal2.columns = ['listing_id', 'num_bookings']
# cleanCal = cleanCal.merge(cleanCal2)
# cleanCal['total_revenue'] = cleanCal['total_revenue'].fillna(0)
# cleanCal['revenue_per_booking'] = cleanCal['total_revenue']/cleanCal['num_bookings']
# cleanCal['revenue_per_booking'] = cleanCal['revenue_per_booking'].fillna(0)
# cleanCal

DataError: No numeric types to aggregate

In [11]:
#Draft 2

ex = calendar[calendar['listing_id'] == 3075044].sort_values(['date'])
ex['day_of_week'] = [d.dayofweek for d in ex['date']]
ex_available = ex[ex['available'] == 't']
ex_mean_prices = ex_available.groupby('day_of_week', as_index=True)['price'].mean()
ex = ex.price.fillna(value = ex.day_of_week.apply(lambda x: ex_mean_prices[int(x)]))
ex[726]



AttributeError: 'str' object has no attribute 'dayofweek'

In [14]:
calendar = temp_cal.copy()
temp_cal = calendar.copy()

In [15]:
calendar['modeled_prices'] = np.nan

In [1]:
#for each unique listing id
#create mean_prices.groupby

#n=0
print('hi')
# for list_id in calendar['listing_id'].unique():
#     ex = calendar[calendar['listing_id'] == 3075044].copy()
#     ex['day_of_week'] = [d.dayofweek for d in ex['date']]
#     ex_available = ex[ex['available'] == 't']
#     ex_mean_prices = ex_available.groupby('day_of_week', as_index=True)['price'].mean()
#     ex = ex.price.fillna(value = ex.day_of_week.apply(lambda x: ex_mean_prices[int(x)]))
#     print(list_id)

#     for i in ex.index:
#         calendar['modeled_prices'][i] = ex[i]
    
calendar

hi


NameError: name 'calendar' is not defined

In [ ]:
#Rob
#Initial Master Merging Merging by IDs between cal and "features" df
listingRev = features.merge(cleanCal, how = 'left', left_on='id', right_on='listing_id')
listingRev

In [ ]:
#Neighborhood Summary
#The super groupby by neighborhood. Collin feel free to replace neighbourhood_cleased with mainzip for zip

#The valuable columns I chose were the revenue metrics, # of reviews, beds, baths, review scores, and then amenities
nbr1 = listingRev.groupby('neighbourhood_cleansed', as_index = False)['total_revenue', 'num_bookings', 'number_of_reviews'].sum().sort_values(['total_revenue'])
nbr2 = listingRev.groupby('neighbourhood_cleansed', as_index = False)['accommodates', 'bathrooms', 'bedrooms',
                        'beds', 'review_scores_rating', 'review_scores_accuracy','review_scores_cleanliness', 
         'review_scores_checkin', 'review_scores_communication', 'review_scores_location',
                        'review_scores_value', ].mean().sort_values(['review_scores_value'])

amenityList = ['24-Hour Check-in', 'Air Conditioning', 'Breakfast', 'Buzzer/Wireless Intercom', 'Cable TV', 'Carbon Monoxide Detector','Cat(s)', 'Dog(s)', 'Doorman','Essentials', 'Family/Kid Friendly', 'Fire Extinguisher', 'First Aid Kit', 'Free Parking on Premises', 'Free Parking on Street', 'Gym', 'Hair Dryer', 'Heating', 'Hot Tub','Indoor Fireplace', 'Internet', 'Iron', 'Kitchen','Laptop Friendly Workspace', 'Lock on Bedroom Door', 'Other pet(s)','Paid Parking Off Premises', 'Pets Allowed','Pets live on this property', 'Pool','Smoking Allowed', 'Suitable for Events', 'TV','Washer / Dryer', 'Wheelchair Accessible','Wireless Internet']

dummy1 = listingRev.groupby('neighbourhood_cleansed', as_index = False)[amenityList].sum()
dummy2 = listingRev.groupby('neighbourhood_cleansed', as_index = False)[amenityList].count()                                                              
dummy = dummy1[list(range(1,36))]/dummy2[list(range(1,36))]
dummy['neighbourhood_cleansed'] = dummy1[[0]]

#Super Merge
#Here's the final summary

nbhdsummary = nbr1
for df in [nbr2, dummy]:
    nbhdsummary = nbhdsummary.merge(df)
    
nbhdsummary.sort_values(['total_revenue'], ascending = False)
nbhdsummary.columns.get_loc('Pets Allowed')

nbhdsummary.ix[:,nbhdsummary.columns.get_loc('24-Hour Check-in'):nbhdsummary.columns.get_loc('Wheelchair Accessible')].plot(kind='bar')

In [ ]:
#understand the datatypes explicitly in the dataset
listings.dtypes
calendar.dtypes
reviews.dtypes


#check column names .columns

#check headers .head()

#check tails .tail()

#listings.columns
listings.tail()
#reviews.head()
#listings['neighborhood_overview'].head(5)


In [ ]:
listings_column_names = pd.DataFrame((listings.columns))
listings_column_names_list = list(listings.columns)
object_first_observation = [listings[x].head(1) for x in listings_column_names_list]
observations = pd.DataFrame(object_first_observation)

In [ ]:
observations.to_csv('datasample.csv')

In [ ]:
reviews['listing_id'].value_counts().plot(kind='hist', figsize = (20,10), fontsize = 16, title = 'Frequency of Reviews')
axes = plt.gca()
axes.set_xlim([0,250])
axes.set_ylim([0,2500])
plt.savefig('freq reviews.jpg')
#multi_reviews = (reviews['listing_id'].value_counts() > 1)
#multi_reviews.value_counts().plot(kind='bar')
#data = go.Histogram(x = multi_reviews)
#py.iplot(data, filename = 'reviewsmorethan1')

In [ ]:
#listings['zipcode'].value_counts().plot(kind = 'bar')
zipcodes_listings = listings['mainzip'].value_counts()
zipcodes_listings.plot()

In [ ]:
#plt.scatter(x = listings['host_response_time'], y = listings['review_scores_value'])
listings['host_response_time'].value_counts()
listings['host_response_time_categories'] = np.nan
listings.loc[listings['host_response_time'] == 'within an hour', 'host_response_time_categories'] = 0
listings.loc[listings['host_response_time'] == 'within a few hours', 'host_response_time_categories'] = 1
listings.loc[listings['host_response_time'] == 'within a day', 'host_response_time_categories'] = 2
listings.loc[listings['host_response_time'] == 'a few days or more', 'host_response_time_categories'] = 3

#df.loc[df['Sex'] == 'M', 'Sex_int'] = 0
#df.loc[df['Sex'] == 'F', 'Sex_int'] = 1

In [ ]:
plt.scatter(x = listings['host_response_time_categories'], y = listings['review_scores_value'])

In [ ]:
#plt.scatter(x= listings['zipcode'],y= listings['price'])
#plt.figsize()
#plt.xlabel('zipcode') #,fontsize = )
#plt.ylabel('price')
#fig.savefig('scatter.jpg')
listings['host_response_time'].value_counts()

In [ ]:
#listings.corr().to_csv('listings_correlations.csv')

In [ ]:
split_amenities = listings['amenities'].str.split(',')
list_split_amenities = list(split_amenities)
new_list_split_amenities = list()
for i in list_split_amenities:
    for j in i:
        new_list_split_amenities.append(j)



In [ ]:
#len(new_list_split_amenities)
reduced_new_list_split_amenities = list(set(new_list_split_amenities))
len(reduced_new_list_split_amenities)

In [ ]:
#THIS CODE IS INTENDED TO FIND THE REVENUE 
listings_unique_ids = list(listings['id'].unique())
len(listings_unique_ids)


In [ ]:
#calendar.plot(kind ='panel')
#calendar.set_index = calendar['listing_id']

#sorted(calendar, key = lambda x: (x['listing_id'], x['date']))
calendar.groupby(['listing_id', 'date']).head()
calendar.sort_index(ascending = False)

#Capture median values
calendar[calendar['listing_id'] == 14504422]
count_for_id = calendar[(calendar['available'] == 'f') & (calendar['listing_id'] == 14504422)]['available'].count() 
mean_price =  calendar[(calendar['available'] == 't') & (calendar['listing_id'] == 14504422)]['price'].mean()

revenue_wo_cleaning = mean_price * count_for_id
revenue_wo_cleaning

In [ ]:
for i in 
count_for_id = calendar[(calendar['available'] == 'f') & (calendar['listing_id'] == 14504422)]['available'].count() 
mean_price =  calendar[(calendar['available'] == 't') & (calendar['listing_id'] == 14504422)]['price'].mean()